In [31]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import torch.nn.functional as F
import os

In [32]:
# random seed --> reproducibility
import random
random.seed(42)
torch.manual_seed(42)

In [33]:
# transforms
data_transforms = {
    'train': transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))]),

    'test': transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
}

In [34]:
# import data
data_dir = '/content/drive/MyDrive/chest_xray'
sets = ['train','test']
image_datasets = {x:datasets.ImageFolder(os.path.join(data_dir, x),
                                         data_transforms[x])
                  for x in ['train', 'test']}    

In [35]:
image_datasets

{'train': Dataset ImageFolder
     Number of datapoints: 5216
     Root location: /content/drive/MyDrive/chest_xray/train
     StandardTransform
 Transform: Compose(
                Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
                RandomHorizontalFlip(p=0.5)
                ToTensor()
                Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
            ),
 'test': Dataset ImageFolder
     Number of datapoints: 624
     Root location: /content/drive/MyDrive/chest_xray/test
     StandardTransform
 Transform: Compose(
                Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
                RandomHorizontalFlip(p=0.5)
                ToTensor()
                Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
            )}

In [36]:
train_dataset = image_datasets['train']
test_dataset = image_datasets['test']

In [37]:
train_dataset

Dataset ImageFolder
    Number of datapoints: 5216
    Root location: /content/drive/MyDrive/chest_xray/train
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

In [38]:
# create DataLoader object
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size = 32,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset,
                                           batch_size = 32,
                                           shuffle=False)

In [ ]:
## Size evaluation 
iterator = iter(train_loader)
sample, lable = next(iterator)
sample

In [40]:
sample.shape

torch.Size([32, 3, 224, 224])

In [67]:
conv1 = nn.Conv2d(3,6,25,stride=2) #first cnn
maxpool = nn.MaxPool2d(2,2) #maxpool
conv2 = nn.Conv2d(6,16,25,stride=2) #second cnn

#cnn layer1
x = conv1(sample)
print(x.shape)
x = maxpool(x)
print(x.shape)

#cnn layer2
x = conv2(x)
print(x.shape)
x = maxpool(x)
print(x.shape)

torch.Size([32, 6, 100, 100])
torch.Size([32, 6, 50, 50])
torch.Size([32, 16, 13, 13])
torch.Size([32, 16, 6, 6])


In [68]:
class LeNet(nn.Module):
  def __init__(self):
    super(LeNet, self).__init__()

    self.conv1 = nn.Conv2d(in_channels=3,
                           out_channels=6,
                           kernel_size=25,
                           stride=2)
    self.conv2 = nn.Conv2d(in_channels=6,
                           out_channels=16,
                           kernel_size=25,
                           stride=2)
    
    self.fc_1 = nn.Linear(16*6*6, 120)
    self.fc_2 = nn.Linear(120, 84)
    self.fc_3 = nn.Linear(84, 2)

  def forward(self, x):
    # conv layer 1
    x = self.conv1(x)
    x = F.max_pool2d(x, kernel_size=2)
    x = F.relu(x)

    # conv layer 2
    x = self.conv2(x)
    x = F.max_pool2d(x, kernel_size=2)
    x = F.relu(x)

    # flatten for input -- linear layer
    x = x.view(-1, 16*6*6)

    # fully connected layer 1
    x = self.fc_1(x)
    x = F.relu(x)

    # fully connected layer 2
    x = self.fc_2(x)
    x = F.relu(x)

    # fully connected layer 3
    x = self.fc_3(x)

    return x

In [69]:
# instance of the model
model = LeNet()

In [70]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 150,846 trainable parameters


In [71]:
# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [72]:
model

LeNet(
  (conv1): Conv2d(3, 6, kernel_size=(25, 25), stride=(2, 2))
  (conv2): Conv2d(6, 16, kernel_size=(25, 25), stride=(2, 2))
  (fc_1): Linear(in_features=576, out_features=120, bias=True)
  (fc_2): Linear(in_features=120, out_features=84, bias=True)
  (fc_3): Linear(in_features=84, out_features=2, bias=True)
)

# training loop


In [73]:
# total iteration
len(train_loader)

163

In [74]:
total_batches = len(train_loader)

for epoch in range(10):
    for i , (images, labels) in enumerate(train_loader):
        
        # forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{10}], step [{i+1}/{total_batches}], loss:{loss.item():.4f}')
            
print('Finished Training')    

Epoch [1/4], step [100/163], loss:0.3324
Epoch [2/4], step [100/163], loss:0.2130
Epoch [3/4], step [100/163], loss:0.0681
Epoch [4/4], step [100/163], loss:0.0126
Epoch [5/4], step [100/163], loss:0.4231
Epoch [6/4], step [100/163], loss:0.0695
Epoch [7/4], step [100/163], loss:0.1367
Epoch [8/4], step [100/163], loss:0.0540
Epoch [9/4], step [100/163], loss:0.0233
Epoch [10/4], step [100/163], loss:0.1059
Finished Training


In [75]:
# save this model
torch.save(model.state_dict(), 'modelLeNet.pth')

In [76]:
# evaluation and testing 
with torch.no_grad():
    n_correct =0
    n_samples =0
    
    for images, labels in test_loader:
        # forward -- softmax prediction 
        outputs = model(images)
        
        # actual prediction
        # value, index
        _, predictions = torch.max(outputs, 1) # multiclass pred
        n_samples += labels.shape[0]
        n_correct += (predictions==labels).sum().item()
        
    acc = 100.0 * n_correct / n_samples
    print(acc)

76.76282051282051


In [96]:
from torchsummary import summary
summary(model,(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 6, 100, 100]          11,256
            Conv2d-2           [-1, 16, 13, 13]          60,016
            Linear-3                  [-1, 120]          69,240
            Linear-4                   [-1, 84]          10,164
            Linear-5                    [-1, 2]             170
Total params: 150,846
Trainable params: 150,846
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.57
Forward/backward pass size (MB): 0.48
Params size (MB): 0.58
Estimated Total Size (MB): 1.63
----------------------------------------------------------------
